<a href="https://colab.research.google.com/github/Marvin2798/lab-langchain-evaluation/blob/main/lab-langchain-evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab | Langchain Evaluation

## Intro

Pick different sets of data and re-run this notebook. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications.

What did you learn? - Let's discuss that in class

## LangChain: Evaluation

### Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [1]:
! pip install --upgrade langchain

In [2]:
!pip install python-dotenv

In [3]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

### Example 1

#### Create our QandA application

In [4]:
!pip install langchain_openai

In [6]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.8 MB/s eta 0:00:00


In [8]:
!pip install langchain

In [10]:
!pip install langchain_huggingface

In [11]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import LLMChain


In [14]:
file = '/content/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [ ]:
# !pip install --upgrade --force-reinstall sentence-transformers

In [17]:
! pip install docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 6.3 MB/s eta 0:00:00


In [18]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})
).from_loaders([loader])

/usr/local/lib/python3.11/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [20]:
import os
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [21]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

#### Coming up with test datapoints

In [22]:
data[10]

Document(metadata={'source': '/content/OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [23]:
data[11]

Document(metadata={'source': '/content/OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

#### Hard-coded examples

In [24]:
from langchain.prompts import PromptTemplate

In [25]:
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field

examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="Examples:\n"
             "1. Query: Do the Cozy Comfort Pullover Set have side pockets?\n"
             "   Answer: Yes\n"
             "2. Query: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?\n"
             "   Answer: The DownTek collection\n"
             "Query: {query}\n"
             "Answer:"
)

# Define the output model
class Answer(BaseModel):
    answer: str = Field(description="The answer to the query")

# Create the output parser
class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Answer:
        # Split the response to get the answer
        answer = text.strip().split("Answer:")[-1].strip()
        return Answer(answer=answer)

# Initialize the LLM
# llm = OpenAI()
llm = ChatOpenAI()

# Create the LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_parser=AnswerOutputParser()
)

# Example query
query = "Is the Cozy Comfort Pullover Set available in different colors?"

# Run the chain
result = llm_chain.run({"query": query})

# Print the result
print(result)


<ipython-input-25-6c6267e67e31>:46: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
<ipython-input-25-6c6267e67e31>:56: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm_chain.run({"query": query})


answer='Yes, it is available in three different colors: grey, navy, and black.'


#### LLM-Generated examples

In [26]:
from langchain.evaluation.qa import QAGenerateChain

In [27]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [28]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

In [29]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/usr/local/lib/python3.11/dist-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [30]:
new_examples[0]

{'qa_pairs': {'query': "What is the weight of each pair of Women's Campside Oxfords?",
  'answer': 'The approximate weight of each pair is 1 lb. 1 oz.'}}

In [31]:
data[0]

Document(metadata={'source': '/content/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [32]:
d_flattened = [data['qa_pairs'] for data in new_examples]
d_flattened

[{'query': "What is the weight of each pair of Women's Campside Oxfords?",
  'answer': 'The approximate weight of each pair is 1 lb. 1 oz.'},
 {'query': 'What are some key features of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'Some key features of the Recycled Waterhog Dog Mat, Chevron Weave include its rugged construction from recycled plastic materials, its ability to help keep dirt and water off floors, its 24 oz. polyester fabric made from 94% recycled materials, rubber backing, exclusive design, thick and thin fibers for scraping dirt and absorbing water, quick drying, resistance to fading, rotting, mildew, and shedding, and the option to use it indoors or outdoors.'},
 {'query': "What features does the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece offer in terms of sun protection and fit?",
  'answer': 'The swimsuit is made of four-way-stretch and chlorine-resistant fabric, has UPF 50+ rated fabric for sun protection, crossover no-slip straps for a

#### Combine examples

In [33]:
# examples += new_example
examples += d_flattened

In [34]:
examples[0]

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'answer': 'Yes'}

In [35]:
qa.invoke(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'}

### Manual Evaluation - Fun part

In [36]:
import langchain
langchain.debug = True

In [37]:
qa.invoke(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 73\nname: Cozy Cuddles Knit Pullover Set\ndescription: Perfect for lounging, this knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out. \n\nSize & Fit \nPants are Favorite Fit: Sits lower on the waist. \nRelaxed Fit: Our most generous fit sits farthest from the body. \n\nFabric & Care \nIn the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features \nRelaxed fit top with raglan sleeves and rounded hem. \nPull-on pants have 

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'}

In [38]:
# Turn off the debug mode
langchain.debug = False

### LLM assisted evaluation

In [39]:
examples += d_flattened

In [40]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What is the weight of each pair of Women's Campside Oxfords?",
  'answer': 'The approximate weight of each pair is 1 lb. 1 oz.'},
 {'query': 'What are some key features of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'Some key features of the Recycled Waterhog Dog Mat, Chevron Weave include its rugged construction from recycled plastic materials, its ability to help keep dirt and water off floors, its 24 oz. polyester fabric made from 94% recycled materials, rubber backing, exclusive design, thick and thin fibers for scraping dirt and absorbing water, quick drying, resistance to fading, rotting, mildew, and shedding, and the option to use it indoors or outdoors.'},
 {'query': "What features does the Infant and Toddler Girls' Coastal Chil

In [41]:
predictions = qa.batch(examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [42]:
predictions

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes',
  'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection',
  'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'},
 {'query': "What is the weight of each pair of Women's Campside Oxfords?",
  'answer': 'The approximate weight of each pair is 1 lb. 1 oz.',
  'result': "The weight of each pair of Women's Campside Oxfords is approximately 1 lb. 1 oz."},
 {'query': 'What are some key features of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'Some key features of the Recycled Waterhog Dog Mat, Chevron Weave include its rugged construction from recycled plastic materials, its ability to help keep dirt and water off floors, its 24 oz. polyester fabric made from 94% recycled materials, rubber backing, excl

In [43]:
from langchain.evaluation.qa import QAEvalChain

In [44]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [45]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [46]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [47]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    # print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set does have side pockets.

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.

Example 2:
Question: What is the weight of each pair of Women's Campside Oxfords?
Real Answer: The approximate weight of each pair is 1 lb. 1 oz.
Predicted Answer: The weight of each pair of Women's Campside Oxfords is approximately 1 lb. 1 oz.

Example 3:
Question: What are some key features of the Recycled Waterhog Dog Mat, Chevron Weave?
Real Answer: Some key features of the Recycled Waterhog Dog Mat, Chevron Weave include its rugged construction from recycled plastic materials, its ability to help keep dirt and water off floors, its 24 oz. polyester fabric made from 94% recycled 

### Example 2
One can also easily evaluate your QA chains with the metrics offered in ragas

In [49]:
from langchain_huggingface import HuggingFaceEmbeddings
loader = TextLoader("/content/OutdoorClothingCatalog_1000.csv")
index = VectorstoreIndexCreator(embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'mps'})).from_loaders([loader])


llm = ChatOpenAI(temperature= 0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=index.vectorstore.as_retriever(),
    return_source_documents=True,
)

RuntimeError: PyTorch is not linked with support for mps devices

In [50]:
from langchain_huggingface import HuggingFaceEmbeddings
loader = TextLoader("/content/OutdoorClothingCatalog_1000.csv")
index = VectorstoreIndexCreator(embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})).from_loaders([loader]) # Changed device to 'cpu'


llm = ChatOpenAI(temperature= 0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=index.vectorstore.as_retriever(),
    return_source_documents=True,
)

/usr/local/lib/python3.11/dist-packages/langchain/indexes/vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [51]:
# testing it out

question = "How did New York City get its name?"
result = qa_chain.invoke({"query": question})
result["result"]

'New York City was named after the Duke of York, who later became King James II of England. King Charles II of England granted the land to his brother, the Duke of York, in the 17th century. The city was originally called New Amsterdam when it was a Dutch colony, but it was renamed New York when the English took control of the area.'

In [52]:
result

{'query': 'How did New York City get its name?',
 'result': 'New York City was named after the Duke of York, who later became King James II of England. King Charles II of England granted the land to his brother, the Duke of York, in the 17th century. The city was originally called New Amsterdam when it was a Dutch colony, but it was renamed New York when the English took control of the area.',
 'source_documents': [Document(id='2664d412-10e6-4e3d-90f0-589f4e4737a9', metadata={'source': '/content/OutdoorClothingCatalog_1000.csv'}, page_content='sold and thousands of 5-star reviews, our extra large duffle bag is the one customers trust to go the distance. "'),
  Document(id='f17b4231-f93e-43ba-8ff6-a2ea6ac8f378', metadata={'source': '/content/OutdoorClothingCatalog_1000.csv'}, page_content='585,Women\'s Mountain Peak Full-Zip Jacket,"We\'ve taken our classic anorak design and given it a modern twist with a full-zip style and heritage-inspired details.'),
  Document(id='06975cb2-0aa8-4f12

Now in order to evaluate the qa system we generated a few relevant questions. We've generated a few question for you but feel free to add any you want.

In [53]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
]

eval_answers = [
    "8,804,190",
    "Brooklyn",
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [54]:
examples

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'ground_truths': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'ground_truths': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City

#### Introducing RagasEvaluatorChain

`RagasEvaluatorChain` creates a wrapper around the metrics ragas provides (documented [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)), making it easier to run these evaluation with langchain and langsmith.

The evaluator chain has the following APIs

- `__call__()`: call the `RagasEvaluatorChain` directly on the result of a QA chain.
- `evaluate()`: evaluate on a list of examples (with the input queries) and predictions (outputs from the QA chain).
- `evaluate_run()`: method implemented that is called by langsmith evaluators to evaluate langsmith datasets.

lets see each of them in action to learn more.

In [55]:
result = qa_chain.invoke({"query": eval_questions[1]})
result["result"]

'The borough of Brooklyn in New York City has the highest population.'

In [56]:
key_mapping = {
    "query": "question",
    "result": "answer",
    "source_documents": "contexts"
}

result_updated = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        result_updated[new_key] = result[old_key]


In [57]:
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'The borough of Brooklyn in New York City has the highest population.',
 'contexts': [Document(id='f17b4231-f93e-43ba-8ff6-a2ea6ac8f378', metadata={'source': '/content/OutdoorClothingCatalog_1000.csv'}, page_content='585,Women\'s Mountain Peak Full-Zip Jacket,"We\'ve taken our classic anorak design and given it a modern twist with a full-zip style and heritage-inspired details.'),
  Document(id='f6c05378-731c-482c-b357-ecd9e4bd8d85', metadata={'source': '/content/OutdoorClothingCatalog_1000.csv'}, page_content='Specs\nCapacity: 2 people plus gear. \nTent area: 37.5 sq. ft.\nTent floor dimensions: 7\'6"" x 5\'. \nPeak height: 47"". \nMinimum weight: 5 lb. 6 oz. \nPacked size: 22"" x 8"".'),
  Document(id='2664d412-10e6-4e3d-90f0-589f4e4737a9', metadata={'source': '/content/OutdoorClothingCatalog_1000.csv'}, page_content='sold and thousands of 5-star reviews, our extra large duffle bag is the one custom

In [58]:
# !pip install --no-cache-dir recordclass

In [59]:
# !pip install ragas==0.1.9

In [61]:
!pip install ragas==0.1.9

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [62]:
from ragas.integrations.langchain import EvaluatorChain
# from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)

# create evaluation chains
faithfulness_chain   = EvaluatorChain(metric=faithfulness)
answer_rel_chain     = EvaluatorChain(metric=answer_relevancy)
context_rel_chain    = EvaluatorChain(metric=context_relevancy)
context_recall_chain = EvaluatorChain(metric=context_recall)

/usr/local/lib/python3.11/dist-packages/ragas/metrics/__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.metrics._answer_correctness import AnswerCorrectness, answer_correctness
/usr/local/lib/python3.11/dist-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to impo

1. `__call__()`

Directly run the evaluation chain with the results from the QA chain. Do note that metrics like context_relevancy and faithfulness require the `source_documents` to be present.

In [63]:
# Recheck the result that we are going to validate.
result

{'query': 'Which borough of New York City has the highest population?',
 'result': 'The borough of Brooklyn in New York City has the highest population.',
 'source_documents': [Document(id='f17b4231-f93e-43ba-8ff6-a2ea6ac8f378', metadata={'source': '/content/OutdoorClothingCatalog_1000.csv'}, page_content='585,Women\'s Mountain Peak Full-Zip Jacket,"We\'ve taken our classic anorak design and given it a modern twist with a full-zip style and heritage-inspired details.'),
  Document(id='f6c05378-731c-482c-b357-ecd9e4bd8d85', metadata={'source': '/content/OutdoorClothingCatalog_1000.csv'}, page_content='Specs\nCapacity: 2 people plus gear. \nTent area: 37.5 sq. ft.\nTent floor dimensions: 7\'6"" x 5\'. \nPeak height: 47"". \nMinimum weight: 5 lb. 6 oz. \nPacked size: 22"" x 8"".'),
  Document(id='2664d412-10e6-4e3d-90f0-589f4e4737a9', metadata={'source': '/content/OutdoorClothingCatalog_1000.csv'}, page_content='sold and thousands of 5-star reviews, our extra large duffle bag is the one c

**Faithfulness**

In [75]:
eval_result = faithfulness_chain(result_updated)
eval_result["faithfulness_score"]

KeyError: 'faithfulness_score'

High faithfulness_score means that there are exact consistency between the source documents and the answer.

You can check lower faithfulness scores by changing the result (answer from LLM) or source_documents to something else.

In [69]:
fake_result = result.copy()
fake_result["result"] = "we are the champions"
eval_result = faithfulness_chain(fake_result)
eval_result["faithfulness_score"]

ValueError: Missing some input keys: {'answer', 'question', 'contexts'}

In [72]:
# Recheck the result that we are going to validate.
result

# Extract the page_content from the Document objects in source_documents
result_updated['contexts'] = [doc.page_content for doc in result['source_documents']]

eval_result = faithfulness_chain(result_updated) # Re-compute eval_result here
faithfulness_score_str = str(eval_result["faithfulness_score"])

KeyError: 'faithfulness_score'

**Context Relevancy**

In [78]:
eval_result = context_recall_chain(result)
eval_result["context_recall_score"]

ValueError: Missing some input keys: {'ground_truth', 'question', 'contexts'}

High context_recall_score means that the ground truth is present in the source documents.

You can check lower context recall scores by changing the source_documents to something else.

In [77]:
from langchain.schema import Document
fake_result = result.copy()
fake_result["source_documents"] = [Document(page_content="I love christmas")]
eval_result = context_recall_chain(fake_result)
eval_result["context_recall_score"]

ValueError: Missing some input keys: {'ground_truth', 'question', 'contexts'}

2. `evaluate()`

Evaluate a list of inputs/queries and the outputs/predictions from the QA chain.

In [79]:
# run the queries as a batch for efficiency
predictions = qa_chain.batch(examples)

# evaluate
print("evaluating...")
r = faithfulness_chain.evaluate(examples, predictions)
r

evaluating...


AttributeError: 'EvaluatorChain' object has no attribute 'evaluate'

In [76]:
# evaluate context recall
print("evaluating...")
r = context_recall_chain.evaluate(examples, predictions)
r

evaluating...


AttributeError: 'EvaluatorChain' object has no attribute 'evaluate'